In [125]:
import pandas as pd
from sklearn.model_selection import train_test_split

dataframe = pd.read_csv('../w3_s16/fastAPI deploy model/spam_detection.csv')
X = dataframe.drop('label',axis = 1)
X

,num_links,num_words,num_special_chars,has_spammy_keywords
0,4,39,15,1
1,1,72,4,1
2,3,164,5,1
3,3,63,5,1
4,1,122,1,0
...,...,...,...,...
9995,5,15,1,0
9996,1,68,14,1
9997,2,184,7,1
9998,3,132,12,1


In [126]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[ 1.41859681, -1.10969715,  0.96500026,  1.0010005 ],
       [-0.71612953, -0.52802928, -0.94616658,  1.0010005 ],
       [ 0.70702136,  1.09359024, -0.77242414,  1.0010005 ],
       ...,
       [-0.00455408,  1.44611622, -0.42493926,  1.0010005 ],
       [ 0.70702136,  0.52954867,  0.44377294,  1.0010005 ],
       [-0.71612953, -1.62085983, -1.29365146,  1.0010005 ]],
      shape=(10000, 4))

In [127]:
from sklearn.ensemble import IsolationForest
model = IsolationForest(contamination = 0.1, random_state=0)
model.fit(X_scaled)

,"n_estimators n_estimators: int, default=100The number of base estimators in the ensemble.",100
,"max_samples max_samples: ""auto"", int or float, default=""auto""The number of samples to draw from X to train each base estimator.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` samples.- If ""auto"", then `max_samples=min(256, n_samples)`.If max_samples is larger than the number of samples provided,all samples will be used for all trees (no sampling).",'auto'
,"contamination contamination: 'auto' or float, default='auto'The amount of contamination of the data set, i.e. the proportionof outliers in the data set. Used when fitting to define the thresholdon the scores of the samples.- If 'auto', the threshold is determined as in the original paper.- If float, the contamination should be in the range (0, 0.5]... versionchanged:: 0.22 The default value of ``contamination`` changed from 0.1 to ``'auto'``.",0.1
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator.- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.Note: using a float number less than 1.0 or integer less than number offeatures will enable feature subsampling and leads to a longer runtime.",1.0
,"bootstrap bootstrap: bool, default=FalseIf True, individual trees are fit on random subsets of the trainingdata sampled with replacement. If False, sampling without replacementis performed.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for :meth:`fit`. ``None`` means 1unless in a :obj:`joblib.parallel_backend` context. ``-1`` means usingall processors. See :term:`Glossary ` for more details.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseudo-randomness of the selection of the featureand split values for each branching step and each tree in the forest.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",0
,"verbose verbose: int, default=0Controls the verbosity of the tree building process.",0
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fit a wholenew forest. See :term:`the Glossary `... versionadded:: 0.21",False


In [128]:
scores = model.decision_function(X_scaled)
dataframe['anomaly_score'] = scores
dataframe

,num_links,num_words,num_special_chars,has_spammy_keywords,label,anomaly_score
0,4,39,15,1,NOT SPAM,0.035947
1,1,72,4,1,NOT SPAM,0.084360
2,3,164,5,1,NOT SPAM,0.056896
3,3,63,5,1,NOT SPAM,0.077531
4,1,122,1,0,NOT SPAM,0.061485
...,...,...,...,...,...,...
9995,5,15,1,0,NOT SPAM,-0.034700
9996,1,68,14,1,NOT SPAM,0.090820
9997,2,184,7,1,SPAM,0.063758
9998,3,132,12,1,SPAM,0.089205


In [129]:
threshold = pd.Series(scores).quantile(0.02)
dataframe["is_outlier"] = scores < threshold
dataframe


,num_links,num_words,num_special_chars,has_spammy_keywords,label,anomaly_score,is_outlier
0,4,39,15,1,NOT SPAM,0.035947,False
1,1,72,4,1,NOT SPAM,0.084360,False
2,3,164,5,1,NOT SPAM,0.056896,False
3,3,63,5,1,NOT SPAM,0.077531,False
4,1,122,1,0,NOT SPAM,0.061485,False
...,...,...,...,...,...,...,...
9995,5,15,1,0,NOT SPAM,-0.034700,False
9996,1,68,14,1,NOT SPAM,0.090820,False
9997,2,184,7,1,SPAM,0.063758,False
9998,3,132,12,1,SPAM,0.089205,False


In [130]:
dataframe[dataframe.is_outlier].sort_values("anomaly_score").head(10)


,num_links,num_words,num_special_chars,has_spammy_keywords,label,anomaly_score,is_outlier
7464,7,191,0,1,SPAM,-0.105920,True
6855,8,65,0,0,NOT SPAM,-0.105110,True
6419,7,197,5,0,SPAM,-0.103215,True
937,7,180,1,0,SPAM,-0.097766,True
3391,6,190,19,1,SPAM,-0.093481,True
6026,7,185,11,0,SPAM,-0.092704,True
8984,7,13,9,0,NOT SPAM,-0.091643,True
3078,7,189,14,0,SPAM,-0.091031,True
5719,7,181,13,0,SPAM,-0.089515,True
9051,9,57,19,1,SPAM,-0.088718,True


In [131]:
import pickle
import os

if not os.path.exists('models'):
    os.makedirs('models')

with open('models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('models/outlier_detection_model.pkl', 'wb') as file:
    pickle.dump(model, file)

